In [30]:
from __future__ import division
from collections import Counter
from copy import deepcopy
from functools import partial
import random

import string

import numpy as np
import nltk

ALPHABET = string.ascii_lowercase
N_LETTERS = len(ALPHABET)
N_LETTERS

CHARS = string.ascii_lowercase + u' '

PUNCTUATION_REMOVER = {
    ord(c): None 
    for c in string.punctuation + string.digits
}
PUNCTUATION_REMOVER.update({
    ord(u'\xe6'): None,
    ord(u'\xe8'): None,
    ord(u'\xe9'): None,
    ord(u'\xee'): None,
    ord(u'\x1a'): None
})

LETTER_TO_INDEX = {unicode(s): i for i, s in enumerate(string.ascii_lowercase + u' ')}

def wordify(i):
    return CHARS[i]
def numbrify(msg):
    return [LETTER_TO_INDEX[m] for m in msg]

wordify = np.vectorize(wordify)

In [41]:
def clean(s):
    return " ".join(s.translate(PUNCTUATION_REMOVER).lower().split())

class Cipher(object):
    chars = unicode(string.ascii_lowercase + ' ')
    ords = tuple(ord(c) for c in chars)
    n_chars = len(chars)
    
    def __init__(self, encoder):
        self._encoder = {ord(k): ord(v) for k, v in encoder.iteritems()}
        self._decoder = {ord(v): ord(k) for k, v in encoder.iteritems()}
        
    @classmethod
    def get_random_cipher(cls):
        c = Cipher({c: c for c in cls.chars})
        return c.mutate_key(27)
    
    @property
    def encoder(self):
        return deepcopy(self._encoder)
    
    @property
    def encoder_matrix(self):
        m = np.zeros((self.n_chars, self.n_chars))
    
    @property
    def decoder(self):
        return deepcopy(self._decoder)
    
    def encipher(self, msg):
        msg = clean(unicode(msg))
        return msg.translate(self._encoder)
    
    def decipher(self, msg):
        return msg.translate(self._decoder)
    
    def mutate_key(self, n_swaps):
        to_swap = random.sample(self.ords, n_swaps)
        candidates = [self.encoder[c] for c in to_swap]
        random.shuffle(candidates)
        self._encoder.update(dict(zip(to_swap, candidates)))
        self._decoder = {v: k for k, v in self._encoder.iteritems()}
        return self
        
c = Cipher.get_random_cipher()
print c.encipher('hello world')
print c.decipher(c.encipher('hello world'))

znppbiabypk
hello world


In [32]:
corpus = " ".join(str(clean(nltk.corpus.gutenberg.raw())).lower().split())
# with open('gutenberg.txt', 'w') as f:
#     f.write(corpus)

In [33]:
letter_counts = Counter(corpus)
letters = sorted(letter_counts.keys())
n_letters = len(letters)
letter_counts

Counter({' ': 2102545,
         'a': 731203,
         'b': 139846,
         'c': 185849,
         'd': 400494,
         'e': 1119617,
         'f': 209239,
         'g': 172048,
         'h': 650743,
         'i': 577691,
         'j': 15946,
         'k': 66676,
         'l': 375313,
         'm': 230032,
         'n': 615091,
         'o': 678136,
         'p': 136173,
         'q': 7552,
         'r': 502402,
         's': 556863,
         't': 827161,
         'u': 252211,
         'v': 83829,
         'w': 201292,
         'x': 9160,
         'y': 176040,
         'z': 5525})

In [135]:
init_letter_probs = np.array([letter_counts[l] for l in letters])
init_letter_probs = np.log(init_letter_probs / init_letter_probs.sum())

def get_trans(n_elements, element_to_index, corpus, order=1, smoothing_factor=1):
    """
    Returns a matrix of log transition probabilities, T, where T[ix] is the 
    conditional probability of seeing element ix[-1] after sequence ix[:-1]
    """
    trans = np.ones((n_elements,)*(1 + order)) * smoothing_factor 
    
    for i in xrange(len(corpus) - order):
        ix = tuple(element_to_index[e] for e in corpus[i: i + order + 1])
        trans[ix] += 1

    return np.log(trans / trans.sum(axis=1, keepdims=True))

letter_trans_probs = get_trans(n_letters, LETTER_TO_INDEX, corpus, order=1)

In [118]:
def get_likelihood(message, init_probs, trans_probs, element_to_index):
    """
    Returns the likelihood of a given message according
    to probabilities init_probs and trans_probs, where element_to_index
    converts from characters (or words if message is a list of words) to 
    array indicies.
    """
    order = trans_probs.ndim - 1
    prob = 0
    for i in xrange(len(message)):
        if i < order:
            prob += init_probs[element_to_index[message[i]]] 
        else: 
            ix = tuple(element_to_index[e] for e in message[i - order: i + 1])
            prob += trans_probs[ix]
    return prob

In [119]:
str_likelihood = lambda x: get_likelihood(x, init_letter_probs, letter_trans_probs, LETTER_TO_INDEX)
get_likelihood('car', init_letter_probs, letter_trans_probs, LETTER_TO_INDEX)

-13.311504437633705

In [120]:
def get_mapping():
    m = np.identity(27)
    np.random.shuffle(m)
    return m

def matrix_to_dict(x):
    return {CHARS[i]: CHARS[x[i].argmax()] for i in range(len(x))}

def dict_to_matrix(d):
    size = len(d)
    m = np.zeros((size, size))
    for k, v in d.iteritems():
        m[LETTER_TO_INDEX[k]][LETTER_TO_INDEX[v]] = 1
    return m

In [121]:
wordify(x.argmax(axis=0))

array([u'p', u'x', u'd', u't', u'b', u'r', u'o', u'k', u'q', u'f', u' ',
       u's', u'z', u'u', u'a', u'm', u'n', u'h', u'w', u'e', u'c', u'v',
       u'g', u'j', u'l', u'y', u'i'], 
      dtype='<U1')

In [132]:
def jumble(order, L):
    """
    order is an array of indices, which correspond to points
    from points1. L is the number of indices to swap
    """
    if L > len(order):
        return np.random.permutation(order)
    
    new_order = order.copy()
    idx = np.arange(order.shape[0])
    swappers = np.random.choice(idx, size=L, replace=False)
    backwards = list(reversed(swappers))
    new_order[swappers], new_order[backwards] = new_order[backwards], new_order[swappers]
    return new_order

def make_new_proposal(s, order, L):
    """
    """
    new_order = jumble(order, L)
    return s, new_order

def sa(start_params, loss_fn, trans_fn, t0=30, thermostat=5, reanneal=300, n=1000):
    """
    As generic a simulated annealing implementation as I could 
    come up with. This actually tries to maximize loss_fn over the params
    """
    prev_val = loss_fn(*start_params)
    best_val = prev_val
    prev_params = start_params
    best_params = start_params
    vals = []
    params = []
    Ls = []
    t = t0
    for i in range(n):
        t *= 0.97
        L = max(np.floor(np.sqrt(t)).astype(int), 1)
        proposed = trans_fn(*prev_params, L=L)
#         print "POOP", proposed
        new_val = loss_fn(*proposed)
#         print "PREV VAL", prev_val
#         print "NEW VAL:", new_val
        delta_val = new_val - prev_val
        vals.append(new_val)
        if delta_val > 0 or np.exp(delta_val / t) > np.random.rand():
            params.append(proposed)
            prev_val = new_val
            prev_params = proposed
            if new_val > best_val:
                best_val = new_val
                best_params = proposed
        if i % reanneal == 0:
            t *= thermostat
        if t < 0.01: 
            t = t0
    return best_params, best_val, params, vals

In [157]:
order = np.arange(len(CHARS))
def likelihood(s, order):
    guess = s.translate(order_to_dict(order))
#     print "GUESS:", guess, str_likelihood(guess) 
    return str_likelihood(guess)
    
best_params, best_val, params, vals = sa([code_msg, order], likelihood, make_new_proposal, n=100000)

In [155]:
code_msg.translate(order_to_dict(best_params[1]))

u'hele os a mod rind seclet kessade that o ak tyfond pil the fblfises ip enclyftond woth ky tif seclet enclyftoin ardilothk on ilgel that ni ine worr me amre ti leag ot bnress they haue sike lear tif nitch cige mleavond techniridy ip whoch o ak nit awale on whoch case ky seclets worr me stiren ang o worr me uely sag ang ky euor frans worr haue meen thwalteg ih what a shake what a hillomre shake that wibrg me si ret bs hife that ot gies nit haffen'

In [57]:
"".join(wordify(np.arange(27)))

u'abcdefghijklmnopqrstuvwxyz '

In [156]:
msg = unicode(open('message.txt').read())
code_msg = c.encipher(msg)

In [110]:
def order_to_dict(order):
    return dict(zip(map(ord, wordify(order)), map(ord, CHARS)))

In [102]:
order_to_dict(np.arange(27))

{u' ': u' ',
 u'a': u'a',
 u'b': u'b',
 u'c': u'c',
 u'd': u'd',
 u'e': u'e',
 u'f': u'f',
 u'g': u'g',
 u'h': u'h',
 u'i': u'i',
 u'j': u'j',
 u'k': u'k',
 u'l': u'l',
 u'm': u'm',
 u'n': u'n',
 u'o': u'o',
 u'p': u'p',
 u'q': u'q',
 u'r': u'r',
 u's': u's',
 u't': u't',
 u'u': u'u',
 u'v': u'v',
 u'w': u'w',
 u'x': u'x',
 u'y': u'y',
 u'z': u'z'}